In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from collections import Counter

In [2]:
#This functione sorts the dataframe column and then add sequece number to dataframe
def give_sort_number(df,column_name,ascending_order=True):
    df = df.sort_values(by=[column_name], ascending=ascending_order).reset_index()
    df.index.name = "Sort_"+column_name
    df = df.reset_index()
    df["Sort_"+column_name] += 1
    return df

In [3]:
# mainDF = pd.read_excel("LabHoursData.xlsx")
# mainDF.columns

In [4]:
mainDF = pd.read_excel("Data//October_10//LabHoursData.xlsx")
mainDF.columns

Index(['#', 'Machine Type', 'Machine', 'Wing', 'Floor', 'Run No', 'Pcs',
       'StartDate', 'StartTime', 'StopDate', 'StopTime', 'Run %',
       'LateStartReason', 'Stop Reasons', 'Recipe', 'Run Intensity',
       'Ini Category', 'Grown Category', 'RunStatus', 'IniAvgHeight',
       'ProdHeight', 'GrownHeight', 'GrowingHours', 'BreakdownTime',
       'From To Hours', 'Actual Hours', 'RunHours', 'IniWeight', 'FinalWeight',
       'GrownWeight', 'Width', 'Length', 'Dmtr', 'Unit', 'Month', 'GrRate'],
      dtype='object')

In [5]:
mainDF.tail(5)

,#,Machine Type,Machine,Wing,Floor,Run No,Pcs,StartDate,StartTime,StopDate,...,RunHours,IniWeight,FinalWeight,GrownWeight,Width,Length,Dmtr,Unit,Month,GrRate
5300,NaN,New,AG42,NaN,G,AG42-6,14,2022-09-25 14:30:00,2022-09-25 14:30:00,2022-10-03 12:30:00,...,190.0,6.21690,30.915960,24.699060,10.084285,11.525714,45.37,A,2022-10 (Oct),12.42
5301,NaN,New,AG42,NaN,G,AG42-7,12,2022-10-04 01:00:00,2022-10-04 01:00:00,2022-10-13 05:00:00,...,220.0,42.41000,133.955000,91.545000,9.824166,13.041666,44.29,A,2022-10 (Oct),12.95
5302,NaN,New,AG42,NaN,G,AG42-8,16,2022-10-14 05:30:00,2022-10-14 05:30:00,2022-10-24 05:30:00,...,240.0,12.06000,99.820000,87.760000,8.600000,10.553125,43.09,A,2022-10 (Oct),12.13
5303,NaN,New,AG42,NaN,G,AG42-9,15,2022-10-24 17:00:00,2022-10-24 17:00:00,2022-11-01 17:00:00,...,192.0,28.35032,87.578985,59.228665,8.541333,10.338000,41.13,A,2022-10 (Oct),12.14
5304,NaN,NaN,NaN,NaN,NaN,Count: 5304,Sum: 54611,NaT,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Avg: 12.51


In [6]:
#remove = "BreakdownTime", "GrowingHours"
final_DF = mainDF[["Machine Type", "Machine", "GrownWeight", "ProdHeight", "Dmtr", 
                   "Actual Hours", "GrRate", "Stop Reasons", "LateStartReason"]]
#final_DF[final_DF.isna().any(axis=1)]
final_DF = final_DF.rename(columns = {'Machine Type':'Machine_Type', 'Stop Reasons':'Stop_Reasons', 'Actual Hours':'Actual_Hours'})

final_DF.tail()

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
5300,New,AG42,24.699060,0.962177,45.37,190.0,12.42,"TIDE O , N/A",NL T
5301,New,AG42,91.545000,4.398333,44.29,220.0,12.95,"TIDE O , N/A",NL T
5302,New,AG42,87.760000,3.350625,43.09,240.0,12.13,"TIDE O , N/A",HLM LT
5303,New,AG42,59.228665,3.358553,41.13,192.0,12.14,"TIDE O , N/A",ON TIME
5304,NaN,NaN,NaN,NaN,NaN,NaN,Avg: 12.51,NaN,NaN


In [7]:
nullID = final_DF[final_DF[['Machine_Type','Machine','GrownWeight', 'ProdHeight','Dmtr', \
                            'Actual_Hours', 'GrRate']].isna().any(axis=1)].index
nullID

Int64Index([5304], dtype='int64')

In [8]:
final_DF = final_DF.drop(nullID)
final_DF

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
0,Old,CA2,97.870000,3.777692,50.71,180.0,13.67,"TIDE O , V LEAK",ON TIME
1,Old,CA2,76.910000,4.452222,46.92,160.0,13.56,"TIDE O , V LEAK",ON TIME
2,Old,CA2,52.035000,3.850909,46.66,110.1,14.25,"TIDE O , N/A",ON TIME
3,Old,CA2,62.470000,3.427777,44.09,136.1,15.2,"TIDE O , N/A",ON TIME
4,Old,CA2,76.228000,2.731699,52.00,200.0,13.45,"TIDE O , V LEAK",ON TIME
...,...,...,...,...,...,...,...,...,...
5299,New,AG30,9.157800,0.381180,44.00,180.0,12.78,"TIDE O , N/A",ON TIME
5300,New,AG42,24.699060,0.962177,45.37,190.0,12.42,"TIDE O , N/A",NL T
5301,New,AG42,91.545000,4.398333,44.29,220.0,12.95,"TIDE O , N/A",NL T
5302,New,AG42,87.760000,3.350625,43.09,240.0,12.13,"TIDE O , N/A",HLM LT


In [9]:
def count_words(df_temp,column_name):
    df_temp[column_name] = df_temp[column_name].str.rstrip('')
    df_temp[column_name] = df_temp[column_name].str.lstrip('')

    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(',,',',')

    df_temp[column_name] = df_temp[column_name].str.replace('\' ','\'')
    df_temp[column_name] = df_temp[column_name].str.replace(' \'','\'')

    df_temp[column_name] = df_temp[column_name].str.rstrip(',')
    df_temp[column_name] = df_temp[column_name].str.lstrip(',')

    df_temp[column_name] = df_temp[column_name].str.split(',')
    df_temp[column_name] = df_temp[column_name].apply(lambda x: Counter(x))
    
    df_temp[column_name] = df_temp[column_name].apply(lambda x: dict(x))
    
    return df_temp

In [10]:
#join all string values of Stop_Reasons in groupby
df_Stop_reason = final_DF.groupby(['Machine','Machine_Type'])['Stop_Reasons'].agg(lambda x: ','.join(x.dropna())).reset_index()

#join all string values of LateStartReason in groupby
df_LateStartReason = final_DF.groupby(['Machine','Machine_Type'])['LateStartReason'].agg(lambda x: ','.join(x.dropna())).reset_index()

In [11]:
final_DF.columns

Index(['Machine_Type', 'Machine', 'GrownWeight', 'ProdHeight', 'Dmtr',
       'Actual_Hours', 'GrRate', 'Stop_Reasons', 'LateStartReason'],
      dtype='object')

In [12]:
df_Stop_reason['Stop_Reasons_counts'] = df_Stop_reason['Stop_Reasons']
df_Stop_reason = count_words(df_Stop_reason,'Stop_Reasons_counts')

df_LateStartReason['LateStartReason_counts'] = df_LateStartReason['LateStartReason']
df_LateStartReason = count_words(df_LateStartReason,'LateStartReason_counts')

In [13]:
final_DF.GrownWeight = pd.to_numeric(final_DF.GrownWeight)
final_DF.ProdHeight = pd.to_numeric(final_DF.ProdHeight)
final_DF.Dmtr = pd.to_numeric(final_DF.Dmtr)
final_DF.Actual_Hours = pd.to_numeric(final_DF.Actual_Hours)
final_DF.GrRate = pd.to_numeric(final_DF.GrRate)

In [14]:
#======================================Dmtr sum===============================================

# df_machine_less35 = pd.merge(
#     final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight','BreakdownTime', 'Dmtr'].sum().reset_index(), 
#     final_DF.groupby(['Machine', 'Machine_Type'])['GrowingHours'].sum().reset_index(), 
#     how="left", on=["Machine", "Machine_Type"])

#======================================Dmtr average===============================================

df_machine_less35 = pd.merge(
        final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight'].sum().reset_index(),
        final_DF.groupby(['Machine', 'Machine_Type'])['GrRate', 'Actual_Hours', 'ProdHeight','Dmtr'].mean().reset_index(), 
                                      how="left", on=["Machine", "Machine_Type"])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3148\3236232831.py:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine', 'Machine_Type'])['GrRate', 'Actual_Hours', 'ProdHeight','Dmtr'].mean().reset_index(),


In [15]:
df_machine_less35.head(2)

,Machine,Machine_Type,GrownWeight,GrRate,Actual_Hours,ProdHeight,Dmtr
0,AA1,Old,264.03016,12.855,136.533333,3.069045,41.901667
1,AA10,Old,272.45215,11.910,140.900000,3.240711,44.670000


In [16]:
df_machine_less35.index.name = "ID"
df_machine_less35.reset_index(inplace=True)

In [17]:
df_machine_less35.head(2)

,ID,Machine,Machine_Type,GrownWeight,GrRate,Actual_Hours,ProdHeight,Dmtr
0,0,AA1,Old,264.03016,12.855,136.533333,3.069045,41.901667
1,1,AA10,Old,272.45215,11.910,140.900000,3.240711,44.670000


In [18]:
df_machine_less35["GrownWeight_less_35"] = df_machine_less35['GrownWeight'] - (df_machine_less35["GrownWeight"] * 35 / 100)

In [19]:
df_machine_less35["GrRate_rank"] = df_machine_less35['GrRate'].apply(lambda x: 1 if (x>12.50 and x<=13.00 ) else 0)

In [20]:
df_machine_less35.head(2)

,ID,Machine,Machine_Type,GrownWeight,GrRate,Actual_Hours,ProdHeight,Dmtr,GrownWeight_less_35,GrRate_rank
0,0,AA1,Old,264.03016,12.855,136.533333,3.069045,41.901667,171.619604,1
1,1,AA10,Old,272.45215,11.910,140.900000,3.240711,44.670000,177.093898,0


In [21]:
df_machine_less35.GrownWeight = pd.to_numeric(df_machine_less35.GrownWeight)
df_machine_less35 = df_machine_less35.sort_values(by=['GrownWeight'], ascending=False).reset_index()
df_machine_less35.index.name = "GrownWeight_rank"
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35["GrownWeight_rank"] += 1
df_machine_less35.index.name = "deleteA"

df_machine_less35.Dmtr = pd.to_numeric(df_machine_less35.Dmtr)
df_machine_less35 = df_machine_less35.sort_values(by=['Dmtr'], ascending=False).reset_index()
df_machine_less35.index.name = "Dmtr_rank"
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35["Dmtr_rank"] += 1
df_machine_less35.index.name = "deleteB"

df_machine_less35.Actual_Hours = pd.to_numeric(df_machine_less35.Actual_Hours)
df_machine_less35 = df_machine_less35.sort_values(by=['Actual_Hours'], ascending=False).reset_index()
df_machine_less35.index.name = "ActualHours_rank"
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35["ActualHours_rank"] += 1
df_machine_less35.index.name = "deleteC"

df_machine_less35.ProdHeight = pd.to_numeric(df_machine_less35.ProdHeight)
df_machine_less35 = df_machine_less35.sort_values(by=['ProdHeight'], ascending=False).reset_index()
df_machine_less35.index.name = "ProdHeight_rank"
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35["ProdHeight_rank"] += 1
df_machine_less35.index.name = "deleteD"

In [22]:
df_machine_less35['Total_rank'] = df_machine_less35['GrownWeight_rank'] + df_machine_less35['Dmtr_rank'] + \
                                        df_machine_less35['ActualHours_rank'] + df_machine_less35["ProdHeight_rank"]

In [23]:
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35 = df_machine_less35.drop(["deleteA","deleteB","deleteC","deleteD","index"], axis=1)

In [24]:
df_results = pd.merge(pd.merge(df_machine_less35,df_Stop_reason,how="left", on=["Machine", "Machine_Type"]),df_LateStartReason,how="left", on=["Machine", "Machine_Type"])

In [25]:
df_results.columns

Index(['ProdHeight_rank', 'ActualHours_rank', 'Dmtr_rank', 'GrownWeight_rank',
       'ID', 'Machine', 'Machine_Type', 'GrownWeight', 'GrRate',
       'Actual_Hours', 'ProdHeight', 'Dmtr', 'GrownWeight_less_35',
       'GrRate_rank', 'Total_rank', 'Stop_Reasons', 'Stop_Reasons_counts',
       'LateStartReason', 'LateStartReason_counts'],
      dtype='object')

In [26]:
df_results = df_results[['ID','Machine_Type','Machine','GrownWeight','GrownWeight_less_35','Dmtr','Actual_Hours', 
                         'ProdHeight','GrRate','GrownWeight_rank','ActualHours_rank', 'ProdHeight_rank', 'Dmtr_rank',
                         'GrRate_rank','Total_rank','Stop_Reasons_counts', 'LateStartReason_counts']]

In [27]:
df_results.head(2)

,ID,Machine_Type,Machine,GrownWeight,GrownWeight_less_35,Dmtr,Actual_Hours,ProdHeight,GrRate,GrownWeight_rank,ActualHours_rank,ProdHeight_rank,Dmtr_rank,GrRate_rank,Total_rank,Stop_Reasons_counts,LateStartReason_counts
0,674,New,BAE36,261.61625,170.050563,42.340,181.00,6.263716,11.0650,401,192,1,640,0,1234,"{'TIDE O': 4, 'N/A': 4}","{'ON TIME': 1, 'G1': 1, 'POWER PROBLEM': 1, 'N..."
1,612,New,BAD23,225.45149,146.543469,43.415,180.25,5.538719,10.7075,810,204,2,346,0,1362,"{'TIDE O': 4, 'N/A': 4}","{'ON TIME': 1, 'G1': 1, 'NL T': 1, 'HLM LT': 1}"


In [28]:
df_results.ID += 1
df_results = df_results.set_index("ID")
df_results = df_results.sort_index()


In [29]:
if os.path.exists("final.xlsx"):
    os.remove("final.xlsx")
    
df_results.to_excel("final.xlsx", header=1, index_label="ID")

In [30]:
m,t = 'AC61','Old'
final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)]

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
774,Old,AC61,35.997415,1.668732,47.67,206.0,11.75,"TIDE O , N/A",ON TIME
775,Old,AC61,65.775000,3.065000,41.96,200.0,11.95,"TIDE O , N/A",ON TIME
776,Old,AC61,52.785000,2.777500,38.92,180.0,12.56,"TIDE O , N/A",ON TIME
777,Old,AC61,56.355000,4.095000,41.13,180.0,11.89,"TIDE O , N/A",POWER PROBLEM
778,Old,AC61,9.887940,0.412898,39.59,270.0,12.37,"TIDE O , N/A",MW


In [31]:
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrownWeight.sum())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Actual_Hours.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].ProdHeight.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Dmtr.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrRate.mean())

220.80035499999997
207.2
2.4038259999999996
41.854
12.104
